<a href="https://colab.research.google.com/github/berilcankutlu/RandomForest-MissingValues/blob/main/RandomForest_and_MissingValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RandomForestRegressor ve Missing Values


### 1. Gerekli Kütüphanelerin Yüklenmesi


In [ ]:
# Veri işleme, model eğitimi ve değerlendirme için gerekli kütüphanelerin yüklenmesi
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### 2. Verinin Okunması ve İlk İnceleme


In [ ]:
# Pandas ayarları değiştirilerek veri setindeki tüm sütunların ve satırların görülmesi
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Excel dosyasından veri okunması
dfd = pd.read_excel("veri.xlsx", sheet_name="sheet")

# Orijinal veri setini korumak için bir kopya oluşturulması
df = dfd.copy()

### 3. Veri Keşfi (EDA - Exploratory Data Analysis)


In [ ]:
# Veri setinin temel bilgilerini görüntülemek için fonksiyon tanımlanması
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())

check_df(df)

### 4. Veri Temizleme ve Dönüştürme


In [ ]:
# 'Ana K.N.' sütunu yalnızca sayısal değerlere sahip olacak şekilde filtrelenmesi
df = dfd[pd.to_numeric(dfd['Ana K.N.'], errors='coerce').notna()]
# Eğer tüm değerler tam sayıysa int türüne çevrilmesi ve sıralanması
df['Ana K.N.'] = df['Ana K.N.'].astype(int)
df = df.sort_values(by='Ana K.N.', ascending=True)

### 5. Pivot İşlemi


In [ ]:
# Pivot işlemi ve yeni kolonların isimlendirilmesi
df = df.pivot(index=['Ana K.N.', 'K.K.N', 'Doğ. Tar.', 'Ana yaşı', 'D', 'C'],
                     columns='int', values='CA')

df.columns = [f'CA_{col}' for col in df.columns]
df.reset_index(inplace=True)

### 6. Tarihsel Bilgilerin Ayrıştırılması


In [ ]:
df["Yıl"] = df["Doğ. Tar."].dt.year
df["Ay"] = df["Doğ. Tar."].dt.month
df["Gün"] = df["Doğ. Tar."].dt.day

### 7. CA Değerleri Arasındaki Farkların Hesaplanması


In [ ]:
# Belli günler arasındaki CA değer farkları hesaplanması
df["CA_diff_30_1"] = df["CA_30"] - df["CA_1"]
df["CA_diff_60_30"] = df["CA_60"] - df["CA_30"]
df["CA_diff_90_60"] = df["CA_90"] - df["CA_60"]
df["CA_diff_180_90"] = df["CA_180"] - df["CA_90"]

# Eksik değer içeren satırlar kaldırılması
df.dropna(subset=["CA_diff_30_1", "CA_diff_60_30", "CA_diff_90_60", "CA_diff_180_90"], inplace=True)

### 8. Logaritmik Dönüşüm


In [ ]:
# Belirli sütunlar için log(1 + x) dönüşümü yapılması
cols_to_transform = ["CA_1", "CA_30", "CA_60", "CA_90", "CA_180"]
for col in cols_to_transform:
    df[f"log_{col}"] = np.log1p(df[col])

### 9. Kategorik ve Sayısal Değişkenlerin Belirlenmesi


In [ ]:
# Veri setindeki kategorik ve sayısal değişkenler belirlenmesi
def grab_col_names(df, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optional
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.

    """
    cat_cols = [col for col in df.columns if df[col].dtypes=='0']
    num_but_cat = [col for col in df.columns if df[col].nunique() < cat_th and df[col].dtypes != '0']
    cat_but_car = [col for col in df.columns if df[col].nunique() > car_th and df[col].dtypes == '0']

    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in df.columns if df[col].dtypes!='0']
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {df.shape[0]}")
    print(f"Variables : {df.shape[1]}")
    print(f"cat_cols : {len(cat_cols)}")
    print(f"num_cols : {len(num_cols)}")
    print(f"cat_but_car : {len(cat_but_car)}")
    print(f"num_but_cat : {len(num_but_cat)}")
    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = grab_col_names(df)

### 10. Veri Dönüştürme


In [ ]:
# Kategorik değişkenler one-hot encoding ile dönüştürülmesi
df = pd.get_dummies(df[cat_cols + num_cols], drop_first=True)
df = df.loc[:, ~df.columns.str.contains('K.K.N')]

# Boolean değişkenler sayısal hale getirilmesi
bool_cols = ["D_İ", "C_E"]
df[bool_cols] = df[bool_cols].astype(int)

# Label Encoding işlemi uygulanması
le = LabelEncoder()
df['Ana K.N.'] = le.fit_transform(df['Ana K.N.'])

### 11. Model Eğitimi İçin Veri Hazırlığı


In [ ]:
features = ['Ana K.N.','Yıl', 'Ay', 'CA_diff_30_1', 'CA_diff_60_30', 'CA_diff_90_60', 'CA_diff_180_90', 'log_CA_1', 'log_CA_30', 'log_CA_60', 'log_CA_90', 'log_CA_180', "D_İ", "C_E"]
target = ["CA_30", "CA_60", "CA_90", "CA_180"]
# Eksik olmayan veriler seçilmesi
train_data = df.dropna(subset=target)
X = train_data[features]
y = train_data[target]

# Veri eğitim ve test olarak bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 12. Model Eğitimi ve Değerlendirme


In [ ]:
model = RandomForestRegressor(n_estimators=200, max_depth=10, max_features="sqrt", min_samples_leaf=1, min_samples_split=2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
for i, col in enumerate(target):
    print(f"{col} için Model Performansı: R2 Skoru: {r2_score(y_test[col], y_pred[:, i]):.4f}")
